# Hotel Support Agent - Agent Catalog + Couchbase Tutorial

## Overview

This tutorial demonstrates how to build a hotel support agent using:
- **Agent Catalog**: Tool registration and agent framework
- **Couchbase**: Vector database for semantic search and data storage
- **LangChain**: ReAct agent workflow orchestration
- **OpenAI**: Language models and embeddings

### What You'll Learn
1. Setting up Couchbase infrastructure with vector search
2. Registering tools with Agent Catalog
3. Creating a simple LangChain ReAct agent
4. Implementing hotel search and detail retrieval
5. Vector similarity search for hotel recommendations

### Architecture Overview
```
User Query → LangChain ReAct Agent → Agent Catalog Tools → Couchbase Database
                ↓
Tool Execution → Vector Search → Response Generation
```

## Section 1: Environment Setup and Dependencies

First, let's install the required dependencies and set up our environment.

In [ ]:
# Install required packages (uncomment if not already installed)
# !pip install langchain langchain-openai couchbase python-dotenv langchain-couchbase
# !pip install agentc-core agentc-langchain  # These should be installed from local paths

In [ ]:
# Import required libraries
import getpass
import json
import os
import time
from datetime import timedelta

import agentc
import agentc_langchain
import dotenv
from couchbase.auth import PasswordAuthenticator
from couchbase.cluster import Cluster
from couchbase.management.buckets import CreateBucketSettings
from couchbase.management.search import SearchIndex
from couchbase.options import ClusterOptions
from langchain.agents import AgentExecutor, create_react_agent
from langchain.hub import pull
from langchain_couchbase.vectorstores import CouchbaseVectorStore
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

# Load environment variables
dotenv.load_dotenv(override=True)

print("All dependencies imported successfully!")

### Environment Variables Setup

Configure the required environment variables for Couchbase and OpenAI connections.

In [ ]:
def _set_if_undefined(var: str):
    if os.environ.get(var) is None:
        os.environ[var] = getpass.getpass(f"Please provide your {var}: ")

def setup_environment():
    """Setup required environment variables with defaults."""
    required_vars = ['OPENAI_API_KEY', 'CB_HOST', 'CB_USERNAME', 'CB_PASSWORD', 'CB_BUCKET_NAME']
    for var in required_vars:
        _set_if_undefined(var)
    
    defaults = {
        'CB_HOST': 'couchbase://localhost',
        'CB_USERNAME': 'Administrator', 
        'CB_PASSWORD': 'password',
        'CB_BUCKET_NAME': 'vector-search-testing',
        'INDEX_NAME': 'vector_search_agentcatalog',
        'SCOPE_NAME': 'shared',
        'COLLECTION_NAME': 'agentcatalog'
    }
    
    for key, default_value in defaults.items():
        if not os.environ.get(key):
            os.environ[key] = input(f"Enter {key} (default: {default_value}): ") or default_value

# Setup environment
setup_environment()
print("Environment variables configured successfully!")
print(f"Using bucket: {os.environ['CB_BUCKET_NAME']}")

## Section 2: Couchbase Infrastructure Setup

Now let's establish our Couchbase connection and set up the necessary infrastructure.

In [ ]:
def setup_couchbase_connection():
    """Setup Couchbase cluster connection."""
    try:
        auth = PasswordAuthenticator(os.environ['CB_USERNAME'], os.environ['CB_PASSWORD'])
        options = ClusterOptions(auth)
        cluster = Cluster(os.environ['CB_HOST'], options)
        cluster.wait_until_ready(timedelta(seconds=10))
        print("Successfully connected to Couchbase")
        return cluster
    except Exception as e:
        raise ConnectionError(f"Failed to connect to Couchbase: {str(e)}")

# Establish connection
cluster = setup_couchbase_connection()

In [ ]:
def setup_collection(cluster, bucket_name, scope_name, collection_name):
    """Setup Couchbase bucket, scope and collection."""
    try:
        try:
            bucket = cluster.bucket(bucket_name)
            print(f"Bucket '{bucket_name}' exists")
        except Exception:
            print(f"Creating bucket '{bucket_name}'...")
            bucket_settings = CreateBucketSettings(
                name=bucket_name,
                bucket_type='couchbase',
                ram_quota_mb=1024,
                flush_enabled=True,
                num_replicas=0
            )
            cluster.buckets().create_bucket(bucket_settings)
            time.sleep(5)
            bucket = cluster.bucket(bucket_name)
            print(f"Bucket '{bucket_name}' created successfully")

        bucket_manager = bucket.collections()
        
        scopes = bucket_manager.get_all_scopes()
        scope_exists = any(scope.name == scope_name for scope in scopes)
        
        if not scope_exists and scope_name != "_default":
            print(f"Creating scope '{scope_name}'...")
            bucket_manager.create_scope(scope_name)
            print(f"Scope '{scope_name}' created successfully")

        collections = bucket_manager.get_all_scopes()
        collection_exists = any(
            scope.name == scope_name and collection_name in [col.name for col in scope.collections]
            for scope in collections
        )

        if not collection_exists:
            print(f"Creating collection '{collection_name}'...")
            bucket_manager.create_collection(scope_name, collection_name)
            print(f"Collection '{collection_name}' created successfully")

        collection = bucket.scope(scope_name).collection(collection_name)
        time.sleep(3)

        try:
            cluster.query(f"CREATE PRIMARY INDEX IF NOT EXISTS ON `{bucket_name}`.`{scope_name}`.`{collection_name}`").execute()
            print("Primary index created successfully")
        except Exception as e:
            print(f"Warning: Error creating primary index: {str(e)}")

        print("Collection setup complete")
        return collection
    except Exception as e:
        raise RuntimeError(f"Error setting up collection: {str(e)}")

# Setup collection
collection = setup_collection(
    cluster,
    os.environ['CB_BUCKET_NAME'],
    os.environ['SCOPE_NAME'],
    os.environ['COLLECTION_NAME']
)

### Vector Search Index Setup

Load the vector search index configuration from the existing agentcatalog_index.json file.

In [ ]:
def setup_vector_search_index(cluster, index_definition):
    """Setup vector search index for hotel data."""
    try:
        scope_index_manager = cluster.bucket(os.environ['CB_BUCKET_NAME']).scope(os.environ['SCOPE_NAME']).search_indexes()
        
        existing_indexes = scope_index_manager.get_all_indexes()
        index_name = index_definition["name"]

        if index_name not in [index.name for index in existing_indexes]:
            print(f"Creating vector search index '{index_name}'...")
            search_index = SearchIndex.from_json(index_definition)
            scope_index_manager.upsert_index(search_index)
            print(f"Vector search index '{index_name}' created successfully")
        else:
            print(f"Vector search index '{index_name}' already exists")
    except Exception as e:
        raise RuntimeError(f"Error setting up vector search index: {str(e)}")

# Load index definition from file (same as main.py)
try:
    with open('agentcatalog_index.json', 'r') as file:
        index_definition = json.load(file)
    print("Loaded vector search index definition from agentcatalog_index.json")
    setup_vector_search_index(cluster, index_definition)
except Exception as e:
    print(f"Warning: Error loading index definition: {str(e)}")
    print("Continuing without vector search index...")

## Section 3: Hotel Data and Vector Store

Load our hotel data and set up the vector store for semantic search.

In [ ]:
def load_hotel_data():
    """Load hotel data directly from main.py implementation."""
    try:
        hotels_data = [
            {
                "name": "Grand Palace Hotel",
                "location": "New York City",
                "description": "Luxury hotel in Manhattan with stunning city views, world-class amenities, and exceptional service.",
                "price_range": "$300-$500",
                "amenities": ["Pool", "Spa", "Gym", "Restaurant", "Room Service", "WiFi"],
                "rating": 4.8
            },
            {
                "name": "Seaside Resort",
                "location": "Miami Beach",
                "description": "Beautiful beachfront resort with private beach access, multiple pools, and ocean view rooms.",
                "price_range": "$200-$400",
                "amenities": ["Beach Access", "Pool", "Restaurant", "Bar", "Water Sports", "WiFi"],
                "rating": 4.6
            },
            {
                "name": "Mountain Lodge",
                "location": "Aspen",
                "description": "Cozy mountain lodge perfect for skiing and hiking, featuring rustic charm and mountain views.",
                "price_range": "$150-$300",
                "amenities": ["Ski Access", "Fireplace", "Restaurant", "Hot Tub", "Hiking Trails", "WiFi"],
                "rating": 4.5
            },
            {
                "name": "Business Center Hotel",
                "location": "Chicago",
                "description": "Modern business hotel in downtown Chicago with state-of-the-art conference facilities.",
                "price_range": "$180-$280",
                "amenities": ["Business Center", "Meeting Rooms", "Gym", "Restaurant", "WiFi", "Parking"],
                "rating": 4.3
            },
            {
                "name": "Boutique Inn",
                "location": "San Francisco",
                "description": "Charming boutique hotel in the heart of San Francisco with unique decor and personalized service.",
                "price_range": "$220-$350",
                "amenities": ["Concierge", "Restaurant", "Bar", "WiFi", "Pet Friendly", "Valet Parking"],
                "rating": 4.7
            }
        ]
        
        hotel_texts = []
        for hotel in hotels_data:
            text = f"{hotel['name']} in {hotel['location']}. {hotel['description']} Price range: {hotel['price_range']}. Rating: {hotel['rating']}/5. Amenities: {', '.join(hotel['amenities'])}"
            hotel_texts.append(text)
        
        return hotel_texts
    except Exception as e:
        raise ValueError(f"Error loading hotel data: {str(e)}")

def setup_vector_store(cluster):
    """Setup vector store and load hotel data."""
    try:
        embeddings = OpenAIEmbeddings(
            api_key=os.environ['OPENAI_API_KEY'],
            model="text-embedding-3-small"
        )
        
        vector_store = CouchbaseVectorStore(
            cluster=cluster,
            bucket_name=os.environ['CB_BUCKET_NAME'],
            scope_name=os.environ['SCOPE_NAME'],
            collection_name=os.environ['COLLECTION_NAME'],
            embedding=embeddings,
            index_name=os.environ['INDEX_NAME'],
        )
        
        hotel_data = load_hotel_data()
        
        try:
            vector_store.add_texts(texts=hotel_data, batch_size=10)
            print("Hotel data loaded into vector store successfully")
        except Exception as e:
            print(f"Warning: Error loading hotel data: {str(e)}. Vector store created but data not loaded.")
        
        return vector_store
    except Exception as e:
        raise ValueError(f"Error setting up vector store: {str(e)}")

# Setup vector store
vector_store = setup_vector_store(cluster)

# Display sample data
hotel_texts = load_hotel_data()
print(f"\nLoaded {len(hotel_texts)} hotels:")
for i, text in enumerate(hotel_texts[:3]):
    print(f"{i+1}. {text[:80]}...")

## Section 4: Agent Catalog Integration

Initialize the Agent Catalog and set up tool registration.

In [ ]:
# Initialize Agent Catalog
catalog = agentc.Catalog()
application_span = catalog.Span(name="Hotel Search Agent")

print("Agent Catalog initialized successfully")

## Section 5: Hotel Search Agent Implementation

Set up the LangChain ReAct agent with Agent Catalog tools, copied directly from main.py.

In [ ]:
def setup_llm_and_agent():
    """Setup LLM and create the hotel search agent."""
    try:
        # Setup LLM with Agent Catalog callback
        llm = ChatOpenAI(
            api_key=os.environ['OPENAI_API_KEY'],
            model="gpt-4o",
            temperature=0,
            callbacks=[agentc_langchain.chat.Callback(span=application_span)]
        )
        
        # Load tools from Agent Catalog and convert to LangChain tools
        tool_search = catalog.find("tool", name="search_vector_database")
        tool_details = catalog.find("tool", name="get_hotel_details")
        
        from langchain_core.tools import Tool
        tools = [
            Tool(
                name=tool_search.meta.name,
                description=tool_search.meta.description,
                func=tool_search.func
            ),
            Tool(
                name=tool_details.meta.name, 
                description=tool_details.meta.description,
                func=tool_details.func
            )
        ]
        
        # Create a simple ReAct agent using LangChain
        react_prompt = pull("hwchase17/react")
        agent = create_react_agent(llm, tools, react_prompt)
        agent_executor = AgentExecutor(
            agent=agent, 
            tools=tools, 
            verbose=True, 
            handle_parsing_errors=True
        )
        
        print(f"Hotel search agent created successfully with {len(tools)} tools")
        return agent_executor
        
    except Exception as e:
        print(f"Error setting up agent: {str(e)}")
        raise

# Create the agent
agent_executor = setup_llm_and_agent()

## Section 6: Testing the Hotel Search Agent

Now let's test our hotel search agent with various scenarios.

In [ ]:
def run_hotel_search_query(query: str):
    """Run a hotel search query and display results."""
    print(f"\nHotel Query: {query}")
    print("=" * 50)
    
    try:
        with application_span.new(f"Query: {query}") as span:
            span["query"] = query
            
            # Execute the agent
            response = agent_executor.invoke({"input": query})
            
            # Display results
            output = response.get('output', 'No response generated')
            print(f"\nAgent Response:\n{output}")
            
            span["response"] = output
            return response
            
    except Exception as e:
        error_msg = f"Error processing query: {e}"
        print(f"\nError: {error_msg}")
        return None

print("Hotel Search Agent is ready!")
print("Testing with sample queries...")

In [ ]:
# Test query 1: Luxury hotel search
result = run_hotel_search_query("Find me a luxury hotel with a pool")

In [ ]:
# Test query 2: Beach resort search
result = run_hotel_search_query("I need a beach resort with spa services")

In [ ]:
# Test query 3: Specific hotel details
result = run_hotel_search_query("Get details about Grand Palace Hotel")

In [ ]:
# Test query 4: Business hotel search
result = run_hotel_search_query("Find a business hotel in Chicago with meeting rooms")

In [ ]:
# Test query 5: Mountain lodge search
result = run_hotel_search_query("Show me mountain hotels with ski access")

## Conclusion

You've successfully built a hotel search agent using:

### What We Accomplished
- **Infrastructure Setup**: Configured Couchbase with vector search capabilities
- **Agent Catalog Integration**: Registered and utilized intelligent tools
- **LangChain ReAct Agent**: Built a straightforward agent with tool integration
- **Vector Search**: Enabled semantic hotel search and recommendation
- **Hotel Detail Retrieval**: Implemented comprehensive hotel information lookup

### Key Features Demonstrated
1. **Vector Database Search** - Semantic search for hotel recommendations
2. **Hotel Detail Lookup** - Comprehensive hotel information retrieval
3. **Simple Agent Architecture** - Clean LangChain ReAct agent implementation
4. **Agent Catalog Integration** - Tool registration and usage
5. **Couchbase Vector Store** - Efficient similarity search capabilities

### Further Learning
- [Agent Catalog Documentation](https://docs.agentcatalog.ai)
- [Couchbase Vector Search Guide](https://docs.couchbase.com/server/current/vector-search/vector-search.html)
- [LangChain Documentation](https://langchain.readthedocs.io/)

Thank you for completing this tutorial!